# Debugging Jupyter Notebook for OCTolyzer's analysis of peripapillary OCT B-scans

This notebook copies the step-by-step process of OCTolyzer's analysis pipeline for OCT peripapillary B-scan analysis, and should provide the end-user with the means of debugging the pipeline by testing each step individually.

### Add OCTolyzer to system paths to permit access to analysis files

In [1]:
import sys
from importlib import reload
sys.path.append(r'../')

### Import necessary packages and libraries

In [2]:
import numpy as np
import os
import cv2
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from octolyzer import utils, analyse, analyse_slo, main
from octolyzer.segment.octseg import choroidalyzer_inference, deepgpet_inference
from octolyzer.segment.sloseg import slo_inference, avo_inference, fov_inference
from octolyzer.measure.bscan.thickness_maps import grid, map as map_module

/Users/s1522100/Documents/OCTolyzer/debugging/../octolyzer/segment/octseg/choroidalyzer_inference.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Define the file you want to use to debug, the `path` is the only variable to change

In [3]:
path = r"../demo/input/Peripapillary_1.vol"

In [4]:
fname = os.path.split(path)[1].split(".")[0]

save_path = f"check_peripapillary/{fname}"
segmentation_directory = f"check_peripapillary/oct_segmentations"

if not os.path.exists(os.path.split(save_path)[0]):
    os.mkdir(os.path.split(save_path)[0])

if not os.path.exists(save_path):
    os.mkdir(save_path)

if not os.path.exists(segmentation_directory):
    os.mkdir(segmentation_directory)

In [5]:
choroidalyzer = None
collate_segmentations = True
deepgpet = None

In [6]:
KEY_LAYER_DICT = {"ILM": "Inner Limiting Membrane",
                  "RNFL": "Retinal Nerve Fiber Layer",
                  "GCL": "Ganglion Cell Layer",
                  "IPL": "Inner Plexiform Layer",
                  "INL": "Inner Nuclear Layer",
                  "OPL": "Outer Plexiform Layer",
                  "ELM": "External Limiting Membrane", # Outer nuclear layer
                  "PR1": "Photoreceptor Layer 1",
                  "PR2": "Photoreceptor Layer 2",
                  "RPE": "Retinal Pigment Epithelium",
                  "BM": "Bruch's Membrane Complex", 
                  "CHORupper": "Bruch's Membrane - Choroid boundary",
                  "CHORlower": "Choroid - Sclera boundary"}

## Set the relevant configuration parameters in `param_dict` to whatever is relevant to your analysis

In [7]:
param_dict = {
    "save_individual_segmentations": 1,
    "save_individual_images": 1,
    "preprocess_bscans": 1,
    "analyse_choroid": 1,
    "analyse_slo": 0,
    "custom_maps": [], # this cannot be "0" like it is in config.txt - it is an empty list
    "analyse_all_maps": 1,
    "analyse_square_grid": 0,
    "choroid_measure_type": "perpendicular",
    "linescan_roi_distance": 3000
}

# flags for choroid analysis, preprocessing bscans
preprocess_data = param_dict["preprocess_bscans"]

# For saving out representative Bscan/SLO/segmentation masks
save_ind_segmentations = param_dict["save_individual_segmentations"]
save_ind_images = param_dict["save_individual_images"]

# Custom retinal thickness maps
custom_maps = param_dict["custom_maps"]
all_maps = param_dict["analyse_all_maps"]

# analysing choroid?
analyse_choroid = param_dict['analyse_choroid']

# square grid for Ppole
sq_grid_flag = param_dict['analyse_square_grid']

# analysing SLO?
analyse_slo_flag = param_dict['analyse_slo']

# User-specified measure type for choroid
chor_measure_type = param_dict['choroid_measure_type']

# User-specified ROI distance either side of fovea
macula_rum = param_dict['linescan_roi_distance']

### Load in data from `.vol` file

In [10]:
verbose=1
oct_output = []
logging_list=[]
output = utils.load_volfile(path, preprocess=preprocess_data*analyse_choroid, verbose=verbose,
                            custom_maps=custom_maps, logging=logging_list)
bscan_data, metadata, slo_output, layer_pairwise, logging_list = output
(slo, slo_acq_fixed, slo_acq, (slo_pad_x, slo_pad_y)) = slo_output
N_scans, M, N = bscan_data.shape
slo_N = slo.shape[0]
oct_output.append(bscan_data)

# Pixel spacing, SLO pixel scaling is assumed as isotropic
scaleX, scaleY, scaleZ = metadata["bscan_scale_x"],metadata["bscan_scale_y"],metadata["bscan_scale_z"]
bscan_scale = (scaleX, scaleY)
slo_scale = metadata["slo_scale_xy"]

# Analyse the SLO image
scan_location = metadata["location"]
if scan_location == "peripapillary":
    slo_location = "Optic disc"
else:
    slo_location = "Macula"
eye = metadata["eye"]

output[1]

Reading file Peripapillary_1.vol...
Loaded a peripapillary (circular) B-scan.
Preprocessing by compensating for vessel shadows and brightening choroid...
.vol file contains ILM and BM, but fewer than all inner retinal layer segmentations.
Processing retinal layer segmentations...
Found 3 valid retinal layer segmentations for all B-scans.
Accessing IR-SLO and organising metadata...
Done!


{'Filename': 'Peripapillary_1.vol',
 'eye': 'Right',
 'bscan_type': 'Peripapillary',
 'bscan_resolution_x': 768,
 'bscan_resolution_y': 496,
 'bscan_scale_z': 0.0,
 'bscan_scale_x': 14.721997082233429,
 'bscan_scale_y': 3.8716697599738836,
 'bscan_ROI_mm': 11.306493318114471,
 'scale_units': 'microns_per_pixel',
 'avg_quality': 31.27155113220215,
 'retinal_layers_N': 3,
 'scan_focus': -1.24,
 'visit_date': '2017-01-11T15:23:07',
 'exam_time': '2017-01-11T14:28:17.514830',
 'slo_resolution_px': 768,
 'field_of_view_mm': 8.997421503067017,
 'stxy_coord': '323,344',
 'acquisition_radius_px': 153,
 'acquisition_radius_mm': 1.79,
 'acquisition_optic_disc_center_x': 476,
 'acquisition_optic_disc_center_y': 344,
 'slo_scale_xy': 11.715392582118511,
 'location': 'peripapillary',
 'slo_modality': 'NIR',
 'field_size_degrees': 30}

### Load in SLO models and analyse SLO (assuming the SLO analysis is *not* where the error is coming from)

In [16]:
slo_metrics = False
segmentation_dict = {}
slo_model = slo_inference.SLOSegmenter()
fov_model = fov_inference.FOVSegmenter()
avo_model = avo_inference.AVOSegmenter()
slo_analysis_output = analyse_slo.analyse(255*slo, save_path,
                                        slo_scale, slo_location, eye,
                                        slo_model, avo_model, fov_model,
                                        save_images=save_ind_segmentations, 
                                        compute_metrics=slo_metrics, verbose=verbose, 
                                        collate_segmentations=True, segmentation_dict=segmentation_dict)
slo_meta_df, slo_measure_dfs, _, slo_segmentations, slo_logging_list = slo_analysis_output
logging_list.extend(slo_logging_list)
slo_avimout = slo_segmentations[-1]
fovea_at_slo = slo_meta_df[["slo_fovea_x", "slo_fovea_y"]].values[0].astype(int)



ANALYSING SLO of Peripapillary_1.

SEGMENTING...
    Segmenting binary vessels from SLO image.
    Segmenting fovea from SLO image.
    Segmenting artery-vein vessels and optic disc from SLO image.

Inferring image metadata...
    Location is specified as optic disc-centred.
    Eye type is specified as the Right eye.
Measurements which have units are in microns units. Otherwise they are non-dimensional.

FEATURE MEASUREMENT...
Skipping metric calculation as analyse_slo_flag is 0.


### Load in choroid segmentation models and segment (if `analyse_choroid` is 1)

In [17]:
# Alert to user we are analysing OCT from here on
msg = f"\n\nANALYSING OCT of {fname}.\n"
logging_list.append(msg)
if verbose:
    print(msg)

# Forcing model instantiation if unspecified
# Choroidalyzer for macular B-scans
if choroidalyzer is None or type(choroidalyzer) != choroidalyzer_inference.Choroidalyzer:
    msg = "Loading models..."
    logging_list.append(msg)
    if verbose:
        print(msg)
    choroidalyzer = choroidalyzer_inference.Choroidalyzer()
    
# DeepGPET for peripapillary B-scans
if deepgpet is None or type(deepgpet) != deepgpet_inference.DeepGPET:
    deepgpet = deepgpet_inference.DeepGPET()

# Segment choroid
# If macula-centred, use Choroidalyzer. If optic disc-centred, use deepGPET
scan_type = metadata["bscan_type"]
if analyse_choroid:
    msg = "Segmenting choroid and fovea..."
else:
    msg = "Detecting fovea for grid/ROI alignment (through use of Choroidalyzer)..."
logging_list.append(msg)
if verbose:
    print(msg)

if scan_location == "macular":
    if N_scans == 1 or choroidalyzer.device == 'cpu':
        rvfmasks, foveas, fov_scores = choroidalyzer.predict_list(bscan_data, soft_pred=True)
    else:
        rvfmasks, foveas, fov_scores = choroidalyzer.predict_batch(bscan_data, soft_pred=True)
elif scan_location == "peripapillary":
    rvfmasks = deepgpet.predict_list(bscan_data, soft_pred=True)

# Resolve fovea detection. If at origin then threshold too high, apply filter function and warn user.
if scan_location != "peripapillary":
    fovea_slice_num, fovea, fov_log = utils._get_fovea(rvfmasks, foveas, N_scans, scan_type, logging=[])
    logging_list.extend(fov_log)

# Detect retinal layer keys
pairwise_keys = list(layer_pairwise.keys())
layer_keys = list(set(pd.DataFrame(pairwise_keys).reset_index(drop=True)[0].str.split("_", expand=True).values.flatten()))



ANALYSING OCT of Peripapillary_1.

Segmenting choroid and fovea...


### Save out OCT B-scan segmentations

In [18]:
# Pipeline for peripapillary scan pattern
if scan_location == "peripapillary":
    
    if analyse_choroid:
        traces = utils.get_trace(rvfmasks[0], 0.25, align=True)
        layer_pairwise["CHORupper_CHORlower"] = [np.array(traces)]
        layer_keys.append("CHORupper")
        layer_keys.append("CHORlower")

    layer_keys_copied = layer_keys.copy()
    if save_ind_segmentations:
        fig, (ax0,ax) = plt.subplots(2,1,figsize=(15,10))
        ax0.imshow(bscan_data[0], cmap='gray')
        ax.imshow(bscan_data[0], cmap="gray")
        for key, tr in layer_pairwise.items():
            for (k, t) in zip(key.split("_"), tr[0]):
                if k in layer_keys_copied:
                    ax.plot(t[:,0],t[:,1])
                    layer_keys_copied.remove(k)
        ax.set_axis_off()
        ax0.set_axis_off()
        fig.tight_layout(pad = 0)
        fig.savefig(os.path.join(save_path, f"{fname}_octseg.png"), bbox_inches="tight")
        if collate_segmentations:
            fig.savefig(os.path.join(segmentation_directory, f"{fname}.png"), bbox_inches="tight")
        plt.close()

        msg = f"\nSegmented B-scan visualisation saved out.\n"
        logging_list.append(msg)
        if verbose:
            print(msg)

        # For a single B-scan, measure thickness and area of all layers, and CVI for choroid
    msg = f"""\nMeasuring thickness around the optic disc for retina and/or choroid.
Thickness measurements will be averaged following the standard peripapillary subgrids.
All measurements are made with respect to the image axis (vertical) as this is a circular B-scan (continuous at either end)."""
    logging_list.append(msg)
    if verbose:
        print(msg)


Segmented B-scan visualisation saved out.


Measuring thickness around the optic disc for retina and/or choroid.
Thickness measurements will be averaged following the standard peripapillary subgrids.
All measurements are made with respect to the image axis (vertical) as this is a circular B-scan (continuous at either end).


### Align peripapillary subfield grid with en face fovea and optic disc centre positions (also computing optic disc overlap)

In [19]:
# Extract metadata from SLO for disc-centred inputs to align peripapillary grid
have_slo = 1
if have_slo:    
    od_radius = slo_meta_df.optic_disc_radius_px.values[0].astype(int)

    # Determine A-scan along B-scan which is centred between the fovea and optic-disc.
    # We call this the temporal midpoint
    output = utils.align_peripapillary_data(metadata, fovea_at_slo, slo_acq, slo_avimout, 
                                            fname, save_path, save_ind_segmentations)
    od_centre, offset_ratio, ascan_idx_temp0  = output
    od_overlap = np.round(100*offset_ratio, 3)
    del metadata['stxy_coord']
    msg = f"User-specified optic disc center is {od_overlap}% of the optic disc diameter."
    logging_list.append(msg)
    if verbose:
        print(msg)

    # Add warning to user if optic disc overlap is greater than 15% of the optic disc radius
    if od_overlap > 15:
        od_warning = True
        msg = f"WARNING: This overlap suggests the acquisition is off-centre from the optic disc. Please check scan/optic disc segmentation."
        logging_list.append(msg)
        if verbose:
            print(msg)
    else:
        od_warning = False
        
else:
    fovea_at_slo = np.array([0, 0])
    od_centre = np.array([0, 0])
    od_warning = None
    od_overlap = None
    od_radius = None
    msg = f"WARNING: Without SLO, peripapillary grid will be centred in the middle of the B-scan, and is likely off-centre"
    logging_list.append(msg)
    if verbose:
        print(msg)

    # If not SLO, default alignment for temporal midpoint depends on laterality
    if eye == 'Right':
        ascan_idx_temp0 = N//2
    else:
        ascan_idx_temp0 = 0

User-specified optic disc center is 4.13% of the optic disc diameter.


### Per segmentation layer, measure thickness arrays, average subfield thicknesses and plot+save thickness profiles across peripapillary

In [21]:
# Measure thickness arrays per segmented layer
measure_dict = {}
for key, tr in layer_pairwise.items():

    # Measure thickness across entire B-scan
    peri_refpt = tr[0][0,N//2]
    thickness = map_module.measure_thickness(tr, 
                                             peri_refpt,
                                             bscan_scale, 
                                             offset=0, 
                                             oct_N=N, 
                                             slo_N=slo_N,
                                             measure_type="vertical", 
                                             region_thresh=0,
                                             disable_progress=True)[0][0]
    
    # Pad thickness with zeros if segmentation doesn't extend to entire image
    stx, enx = tr[0][0,[0,-1],0]
    if thickness.shape[0] < N:
        msg = f"\nWARNING: Peripapillary segmentation for layer {key} missing {np.round(100*((stx+(N-enx))/N),2)}% pixels. Interpolating thickness array linearly. Please check segmentation.\n"
        logging_list.append(msg)
        if verbose:
            print(msg)
            
        # pad missing values with NaNs and then wrap array using opposite edges as the thickness array should be continuous at each end
        thickness_padded = np.pad(thickness, (max(0,stx), max(0,(N-1)-enx)), constant_values=np.nan)
        thickness_padded = np.pad(thickness_padded, (N//2,N//2), mode='wrap')

        # Linear inteprolate across NaNs and slice outinterpolated thickness array
        x_grid = np.arange(2*N)
        where_nans = np.isnan(thickness_padded)
        thickness_padded[where_nans]= np.interp(x_grid[where_nans], x_grid[~where_nans], thickness_padded[~where_nans])
        thickness = thickness_padded[N//2:-N//2]

    # Align the thickness vector, depending on laterality
    if eye == 'Right':
        align_idx = N//2 - ascan_idx_temp0
        if align_idx > 0:
            align_thickness = np.pad(thickness, (align_idx, 0), mode="wrap")[:N]
        else:
            align_thickness = np.pad(thickness, (0, -align_idx), mode="wrap")[-align_idx:]
    else:
        align_idx = ascan_idx_temp0 - N//2
        if align_idx > 0:
            align_thickness = np.pad(thickness, (0, align_idx), mode="wrap")[align_idx:]
        else:
            align_thickness = np.pad(thickness, (-align_idx, 0), mode="wrap")[:N]

    # We create a moving average, a smoothed version of the raw aligned thickness values
    ma_idx = 32
    align_thickness_padded = np.pad(align_thickness, (ma_idx,ma_idx), mode="wrap")
    moving_avg = pd.Series(align_thickness_padded).rolling(window=ma_idx, center=True).mean().values[ma_idx:-ma_idx]
    
    # We fit a spline to the raw and smoothed thickness values, and define that over
    # [-180, 180] degree window
    N_line = align_thickness.shape[0]
    x_grid = np.linspace(-180., 180., N_line)
    spline_raw = scipy.interpolate.UnivariateSpline(x_grid, align_thickness)(x_grid)
    spline_ma = scipy.interpolate.UnivariateSpline(x_grid, moving_avg)(x_grid)
    spline_raw_coords = np.concatenate([[x_grid], [spline_raw]]).T
    spline_ma_coords = np.concatenate([[x_grid], [spline_ma]]).T

    # Organise thickness values into their circular subregions
    grid_cutoffs = np.array([0, 45, 90, 135, 225, 270, 315, 360]) - 180
    grids = ["nasal", "infero_nasal", "infero_temporal", 
             "temporal", "supero_temporal", "supero_nasal", "nasal"]
    grid_measures_raw = {g+'_[um]':[] for g in grids}
    grid_measures_ma = {g+'_[um]':[] for g in grids}
    for g_str, g_idx_i, g_idx_j in zip(grids, grid_cutoffs[:-1]+180, grid_cutoffs[1:]+180):
        x_idx_i = int(N*(g_idx_i/360))
        x_idx_j = int(N*(g_idx_j/360))
        grid_measures_raw[g_str+'_[um]'].extend(list(spline_raw_coords[x_idx_i:x_idx_j, 1]))
        grid_measures_ma[g_str+'_[um]'].extend(list(spline_ma_coords[x_idx_i:x_idx_j, 1]))
    
    # Average across entire grid
    grid_measures_raw["All"+'_[um]'] = spline_raw_coords[:,1].mean()
    grid_measures_ma["All"+'_[um]'] = spline_ma_coords[:,1].mean()

    # Average in subgrid of temporal zone, orientated to fovea
    grid_measures_raw["PMB"+'_[um]'] = grid_measures_raw["temporal_[um]"][30:60]
    grid_measures_ma["PMB"+'_[um]'] = grid_measures_ma["temporal_[um]"][30:60]

    # Measure the average thickness per circular subgrid
    grid_means_raw = {key:int(np.mean(value)) for key, value in grid_measures_raw.items()}
    grid_means_ma = {key:int(np.mean(value)) for key, value in grid_measures_ma.items()}

    # Nasal-temporal ratio, catch exception of zero division if segmentation doesn't cover temporal region
    try:
        grid_means_raw["N/T"] = grid_means_raw["nasal_[um]"]/grid_means_raw["temporal_[um]"]
        grid_means_ma["N/T"] = grid_means_ma["nasal_[um]"]/grid_means_ma["temporal_[um]"]
    except:
        grid_means_raw["N/T"] = np.nan
        grid_means_ma["N/T"] = np.nan
    
    # Save to dict
    measure_dict[key] = grid_means_ma

    # Plot the thickness curve, and if SLO show the peripapillary grid measurements overlaid
    if save_ind_images and have_slo:
        grid.plot_peripapillary_grid(slo, slo_acq, metadata, grid_means_ma, fovea_at_slo, 
                                    spline_raw_coords, spline_ma_coords, key, fname+f'_{key}', save_path)
    else:
        grid.plot_thickness_profile(spline_raw_coords, spline_ma_coords, key, fname+f'_{key}', save_path)

# Peripapillary metadata
if od_centre is not None:
    metadata["optic_disc_overlap_index_%"] = od_overlap
    metadata['optic_disc_overlap_warning'] = od_warning
    metadata['optic_disc_x'] = int(od_centre[0])
    metadata['optic_disc_y'] = int(od_centre[1])
    metadata['optic_disc_radius_px'] = od_radius
    metadata["choroid_measure_type"] = 'vertical'

# Add metric units to the end of metadata
metadata["thickness_units"] = "microns"
metadata["choroid_vascular_index_units"] = 'dimensionless'
metadata["choroid_vessel_density_units"] = "micron2"
metadata["area_units"] = "mm2"
metadata["volume_units"] = "mm3"

### Save out image scans

In [22]:
# If saving out bscan and slo image. If ppole, only saying out bscan at fovea
# This is automatically done for AV-line scans.
if save_ind_images:
    if have_slo:
        cv2.imwrite(os.path.join(save_path,f"{fname}_slo.png"), 
                    (255*slo).astype(np.uint8))
    if scan_location != 'peripapillary':
        cv2.imwrite(os.path.join(save_path,f"{fname}_slo_acquisition_lines.png"), 
                    (255*slo_acq).astype(np.uint8))
        cv2.imwrite(os.path.join(save_path,f"{fname}_bscan_fovea.png"), 
                (255*bscan_data[fovea_slice_num]).astype(np.uint8))
    else:
        cv2.imwrite(os.path.join(save_path,f"{fname}_bscan.png"), 
                (255*bscan_data[0]).astype(np.uint8))

### Organise measurements into DataFrame

In [23]:
# For H-line/V-line/Radial organise feature measurements into a DataFrame
if scan_type != "AV-line":
    measure_df = utils.nested_dict_to_df(measure_dict).reset_index()
    if scan_type != 'Peripapillary':
        measure_df = measure_df.rename({"level_0":"scan_number", "level_1":"layer"}, inplace=False, axis=1)
    else:
        measure_df = measure_df.rename({"index":"layer"}, inplace=False, axis=1)
    
    # rename whole/inner/outer retinal layers
    keys_to_names = ['ILM_BM', 'ILM_ELM', 'ELM_BM']
    names_to_keys = ['retina', 'inner_retina', 'outer_retina']
    for k2n, n2k in zip(keys_to_names, names_to_keys):
        measure_df.replace(k2n, n2k, inplace=True)
    
    # order map layer names anatomically
    all_pairwise_layers = list(layer_pairwise.keys())
    all_pairwise_layers = names_to_keys + all_pairwise_layers
    ordered_maps = []
    for map_name in all_pairwise_layers:
        if map_name in list(measure_df.layer.values):
            ordered_maps.append(map_name)
    measure_df['layer'] = pd.CategoricalIndex(measure_df['layer'], ordered=True, categories=ordered_maps)
    measure_df = measure_df.sort_values('layer').reset_index(drop=True)
    measure_dfs = [measure_df]

### Organise segmentations to be saved out

In [24]:
# Layer keys, ordered anaomtically
ordered_keys = np.array(list(KEY_LAYER_DICT))
key_df = pd.DataFrame({"key":layer_keys,"layer":[KEY_LAYER_DICT[key] for key in layer_keys],
                    "layer_number":[np.where(key == ordered_keys)[0][0] for key in layer_keys]})
key_df = key_df.sort_values("layer_number")
del key_df["layer_number"]

In [25]:
# Organise layer segmentations to be saved out - overcomplicated as I am working
# with pairwise segmentation traces, not individual ones. 
seg_df = {}
layer_keys_copied = layer_keys.copy()
for key, trace_xy_all in layer_pairwise.items():
    for k_idx, k in enumerate(key.split("_")):
        if k in layer_keys_copied:
            all_ytr = {}
            for s_idx, trace in enumerate(trace_xy_all):
                t = trace[k_idx]
                (xtr, ytr) = t[:,0], t[:,1]
                try:
                    xst, xen = xtr[[0,-1]]
                    ytr_pad = np.pad(ytr, ((max(xst-1,0), N-xen)), mode="constant")
                    all_ytr[s_idx] = {i:ytr_pad[i] for i in range(N)}
                except Exception as e:
                    message = f"\nAn exception of type {type(e).__name__} occurred. Error description:\n{e.args[0]}"
                    user_fail = f"Failed to store segmentations for B-scan {s_idx+1}/{N_scans} for layer {k}. Saving as NAs"
                    log_save = [message, user_fail]
                    logging_list.extend(log_save)
                    if verbose:
                        print(message)
                        print(user_fail)
                    all_ytr[s_idx] = {i:np.nan for i in range(N)}

            layer_keys_copied.remove(k)
            df = utils.nested_dict_to_df(all_ytr).reset_index()
            df = df.rename({"index":"scan_number"}, inplace=False, axis=1)
            seg_df[k] = df

### Save out results

In [26]:
# Save out core results in an .xlsx file
meta_df = pd.DataFrame(metadata, index=[0])
with pd.ExcelWriter(os.path.join(save_path, f'{fname}_output.xlsx')) as writer:
    
    # Write metadata
    meta_df.to_excel(writer, sheet_name='metadata', index=False)

    # Save out metadata key and descriptions
    metakeydf = utils.metakey_df
    metakeydf = metakeydf[metakeydf.column.isin(list(meta_df.columns))]
    metakeydf.to_excel(writer, sheet_name='metadata_keys', index=False)

    # Write OCT results, either map measurements (for PPole only) or H-line/V-line/Radial measurements
    if scan_type == "Ppole":
        for measure_df, volmeasure_df, grid_type in zip(measure_dfs, volmeasure_dfs, measure_grids):
            measure_df.to_excel(writer, sheet_name=f'{grid_type}_measurements', index=False)
            volmeasure_df.to_excel(writer, sheet_name=f'{grid_type}_volume_measurements', index=False)    
    elif scan_type != "AV-line":
        for measure_df in measure_dfs:
            measure_df.to_excel(writer, sheet_name="oct_measurements", index=False)

    # Write SLO measurements
    if slo_analysis_output is not None and analyse_slo:
        for df in slo_measure_dfs:
            if len(df) > 0:
                z = df.zone.iloc[0]
                df.to_excel(writer, sheet_name=f'slo_measurements_{z}', index=False)

    # Write out segmentations
    for key,df in seg_df.items():
        if key != key.lower():
            name = f"segmentations_{key}"
        else:
            name = f"maps_{key}"
        df.to_excel(writer, sheet_name=name, index=False)

    # write out layer keys
    key_df.to_excel(writer, sheet_name="layer_keys", index=False)

msg = f"\nSaved out metadata, measurements and segmentations."
logging_list.append(msg)
if verbose:
    print(msg)

# Organise outputs from analysis script
oct_measures = measure_dfs.copy()
oct_segmentations_maps = [seg_df]
if scan_location != 'peripapillary':
    oct_segmentations_maps.append(vmasks)
if scan_type == "Ppole":
    oct_segmentations_maps.append(map_dict)
    for df in volmeasure_dfs:
        oct_measures.append(df)
    if sq_grid_flag:
        oct_measures = [oct_measures[0],oct_measures[2],oct_measures[1],oct_measures[3]]
oct_analysis_output = [meta_df, slo, bscan_data] + [oct_measures] + oct_segmentations_maps + [logging_list]

# final log
msg = f"\nCompleted analysis of {fname}.\n"
logging_list.append(msg)
if verbose:
    print(msg)

# Save out log
with open(os.path.join(save_path, f"{fname}_log.txt"), "w") as f:
    for line in logging_list:
        f.write(line+"\n")


Saved out metadata, measurements and segmentations.

Completed analysis of Peripapillary_1.



# Run OCTolyzer's pipeline from scratch on this file to check whether it works properly before/after debugging. 

In [27]:
analyse.analyse(path, save_path="check_peripapillary", param_dict=param_dict)


ANALYSING SLO+OCT OF Peripapillary_1.

Reading file Peripapillary_1.vol...
Loaded a peripapillary (circular) B-scan.
Preprocessing by compensating for vessel shadows and brightening choroid...
.vol file contains ILM and BM, but fewer than all inner retinal layer segmentations.
Processing retinal layer segmentations...
Found 3 valid retinal layer segmentations for all B-scans.
Accessing IR-SLO and organising metadata...
Done!


ANALYSING SLO of Peripapillary_1.

SEGMENTING...
Loading models...
    Segmenting binary vessels from SLO image.
    Segmenting fovea from SLO image.
    Segmenting artery-vein vessels and optic disc from SLO image.

Inferring image metadata...
    Location is specified as optic disc-centred.
    Eye type is specified as the Right eye.
Measurements which have units are in microns units. Otherwise they are non-dimensional.

FEATURE MEASUREMENT...
Skipping metric calculation as analyse_slo_flag is 0.


ANALYSING OCT of Peripapillary_1.

Loading models...
Macular c

((          Filename    location    eye  manual_annotation  slo_fovea_x  \
  0  Peripapillary_1  Optic disc  Right              False           43   
  
     slo_fovea_y  slo_missing_fovea  optic_disc_x  optic_disc_y  \
  0          397              False           470           347   
  
     optic_disc_radius_px      scale measurement_units        scale_units  
  0                    72  11.715393           microns  microns-per-pixel  ,
  [Empty DataFrame
   Columns: []
   Index: []],
  array([[ 70.,  74.,  72., ...,  85.,  83.,  83.],
         [ 71.,  72.,  72., ...,  91.,  88.,  88.],
         [ 73.,  76.,  75., ...,  90.,  90.,  88.],
         ...,
         [ 78.,  77.,  83., ...,  98.,  98.,  97.],
         [ 75.,  79.,  81., ..., 100., 100., 100.],
         [ 73.,  79.,  78., ..., 102., 101., 101.]]),
  [array([[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 